In [ ]:
#tfds.as_dataframe(train_df.head())

!wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
!tar -xvf food-101.tar.gz

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential, load_model, Model
from keras.layers import Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras import applications
from keras.applications import ResNet152V2
from keras.utils.np_utils import to_categorical
from tensorflow.keras import regularizers
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import Adam  
import keras,os,re,math,itertools
import matplotlib.pyplot as plt
import keras.backend as K
from ipywidgets import interact, interactive, fixed
from sklearn.metrics import confusion_matrix
from keras.optimizers import SGD
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
print(tf.config.experimental.list_physical_devices('GPU'))


Using TensorFlow backend.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [2]:
# read data block


# Helper method to split dataset into train and test folders
def prepare_data(filepath, src,dest):
  classes_images = defaultdict(list)
  with open(filepath, 'r') as txt:
      paths = [read.strip() for read in txt.readlines()]
      for p in paths:
        food = p.split('/')
        classes_images[food[0]].append(food[1] + '.jpg')

  for food in classes_images.keys():
    print("\nCopying images into ",food)
    if not os.path.exists(os.path.join(dest,food)):
      os.makedirs(os.path.join(dest,food))
    for i in classes_images[food]:
      copy(os.path.join(src,food,i), os.path.join(dest,food,i))
  print("Copying Done!")

In [ ]:
from collections import defaultdict
import collections
import os
from shutil import copy
from shutil import copytree, rmtree
prepare_data('food-101/meta/train.txt', 'food-101/images', 'train')

In [4]:
prepare_data('food-101/meta/test.txt', 'food-101/images', 'test')


Copying images into  apple_pie

Copying images into  baby_back_ribs

Copying images into  baklava

Copying images into  beef_carpaccio

Copying images into  beef_tartare

Copying images into  beet_salad

Copying images into  beignets

Copying images into  bibimbap

Copying images into  bread_pudding

Copying images into  breakfast_burrito

Copying images into  bruschetta

Copying images into  caesar_salad

Copying images into  cannoli

Copying images into  caprese_salad

Copying images into  carrot_cake

Copying images into  ceviche

Copying images into  cheesecake

Copying images into  cheese_plate

Copying images into  chicken_curry

Copying images into  chicken_quesadilla

Copying images into  chicken_wings

Copying images into  chocolate_cake

Copying images into  chocolate_mousse

Copying images into  churros

Copying images into  clam_chowder

Copying images into  club_sandwich

Copying images into  crab_cakes

Copying images into  creme_brulee

Copying images into  croque_madam

In [5]:
# Check how many files are in the train folder
print("Total number of samples in train folder")
!find train -type d -or -type f -printf '.' | wc -c

# Check how many files are in the test folder
print("Total number of samples in test folder")
!find test -type d -or -type f -printf '.' | wc -c

Total number of samples in train folder
75750
Total number of samples in test folder
25250


In [6]:
# # full image data generator
# tf.keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False,
#     samplewise_center=False,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=False,
#     zca_epsilon=1e-06,
#     rotation_range=0,
#     width_shift_range=0.0,
#     height_shift_range=0.0,
#     brightness_range=None,
#     shear_range=0.0,
#     zoom_range=0.0,
#     channel_shift_range=0.0,
#     fill_mode="nearest",
#     cval=0.0,
#     horizontal_flip=False,
#     vertical_flip=False,
#     rescale=None,
#     preprocessing_function=None,
#     data_format=None,
#     validation_split=0.0,
#     dtype=None,
# )

In [7]:
def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004
lr_scheduler = LearningRateScheduler(schedule)

In [ ]:
strategy = tf.distribute.MirroredStrategy(["GPU:0", "GPU:1", "GPU:2", "GPU:3"])
with strategy.scope():
    # apply functions to the data to improve training
    img_width, img_height = 299, 299
    num_of_classes = 101

    # number of epochs to train top model
    epochs = 20

    # batch size used by flow_from_directory and predict_generator, try 128 if possible
    batch_size = 64
    #important hyper-parameters
    metrics = ['accuracy']

    datagenTrain = ImageDataGenerator(
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False, # randomly flip images
        zoom_range=[.8, 1.2],
        channel_shift_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range = 0.2,
        fill_mode='nearest',
        preprocessing_function = tf.keras.applications.inception_v3.preprocess_input) #preprocess_input is the preprocessing function used in 
                                                    #original resnet152V2 model

    generatorTrain = datagenTrain.flow_from_directory(
        'train',
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

    # iffy about this one
    test_datagen = ImageDataGenerator(zoom_range=[.8,1.2], horizontal_flip=True, vertical_flip=False)

    validation_generator = test_datagen.flow_from_directory(
        'test',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')
    
    
    resnet = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
    x = resnet.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128,activation='relu')(x)
    x = Dropout(0.2)(x)

    predictions = Dense(101,kernel_regularizer=regularizers.l2(0.005), activation='softmax')(x)

    model = Model(inputs=resnet.input, outputs=predictions)
    model.compile(optimizer=SGD(lr=0.02, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    
    checkpointer = ModelCheckpoint(filepath='best_model_3class.hdf5', verbose=1, save_best_only=True)
    csv_logger = CSVLogger('history_3class.log')

    nb_train_samples = 75750
    nb_validation_samples = 25250
    batch_size = 16

#     history = model.fit_generator(generatorTrain,
#                         #steps_per_epoch = nb_train_samples // batch_size,
#                         nb_val_samples = nb_validation_samples,
#                         validation_data=validation_generator,
#                         #validation_steps=nb_validation_samples // batch_size,
#                         samples_per_epoch = nb_train_samples / 25,
#                         epochs=30,
#                         verbose=1,
#                         callbacks=[csv_logger, checkpointer, lr_scheduler])
    history = model.fit(generatorTrain,
                        batch_size= 32,
                        epochs = 30,
                        validation_data = validation_generator,
                        verbose = 1,
                        callbacks = [csv_logger, checkpointer, lr_scheduler],
                        )

    model.save('model_trained_3class.hdf5')



INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Epoch 1/30
INFO:tensorflow:batch_all_reduce: 192 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 192 all-reduces with algorithm = nccl, num_packs = 1
1184/1184 [==============================] - 2248s 2s/step - loss: 3.4806 - accuracy: 0.2971 - val_loss: 55.9542 - val_accuracy: 0.0107

Epoch 00001: val_loss improved from inf to 55.95423, saving model to best_model_3class.hdf5
Epoch 2/30
1184/1184 [==============================] - 2148s 2s/step - loss: 1.8369 - accuracy: 0.5897 - val_loss: 35.9449 - val_accuracy: 0.0096

Epoch 00002: val_loss improved from 55.95423 to 35.94493, saving model to best_model_3class.hdf5
Epoch 